# Geographical and Temporal Aggregation (Tourism)

> Geographical and Temporal Hierarchical Forecasting on Australian Tourism Data

In many applications, a set of time series is hierarchically organized. Examples include the presence of geographic levels, products, or categories that define different types of aggregations. In such scenarios, forecasters are often required to provide predictions for all disaggregate and aggregate series. A natural desire is for those predictions to be **"coherent"**, that is, for the bottom series to add up precisely to the forecasts of the aggregated series.

In this notebook we present an example on how to use `HierarchicalForecast` to produce coherent forecasts between geographical levels. We will use the classic Australian Domestic Tourism (`Tourism`) dataset, which contains monthly time series of the number of visitors to each state of Australia.

We will first load the `Tourism` data and produce base forecasts using an `ETS` model from `StatsForecast`, and then reconciliate the forecasts with several reconciliation algorithms from `HierarchicalForecast`. Finally, we show the performance is comparable with the results reported by the [Forecasting: Principles and Practice](https://otexts.com/fpp3/tourism.html) which uses the R package [fable](https://github.com/tidyverts/fable).

You can run these experiments using CPU or GPU with Google Colab.

<a href="https://colab.research.google.com/github/Nixtla/hierarchicalforecast/blob/main/nbs/examples/AustralianDomesticTourism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install hierarchicalforecast statsforecast

## 1. Load and Process Data

In this example we will use the [Tourism](https://otexts.com/fpp3/tourism.html) dataset from the [Forecasting: Principles and Practice](https://otexts.com/fpp3/) book.

The dataset only contains the time series at the lowest level, so we need to create the time series for all hierarchies.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
Y_df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/tourism.csv')
Y_df = Y_df.rename({'Trips': 'y', 'Quarter': 'ds'}, axis=1)
Y_df.insert(0, 'Country', 'Australia')
Y_df = Y_df[['Country', 'Region', 'State', 'Purpose', 'ds', 'y']]
Y_df['ds'] = Y_df['ds'].str.replace(r'(\d+) (Q\d)', r'\1-\2', regex=True)
Y_df['ds'] = pd.PeriodIndex(Y_df["ds"], freq='Q').to_timestamp()
Y_df.head()

,Country,Region,State,Purpose,ds,y
0,Australia,Adelaide,South Australia,Business,1998-01-01,135.077690
1,Australia,Adelaide,South Australia,Business,1998-04-01,109.987316
2,Australia,Adelaide,South Australia,Business,1998-07-01,166.034687
3,Australia,Adelaide,South Australia,Business,1998-10-01,127.160464
4,Australia,Adelaide,South Australia,Business,1999-01-01,137.448533


## 2. Cross-sectional reconciliation

### 2a. Aggregating the dataset according to cross-sectional hierarchy

The dataset can be grouped in the following non-strictly hierarchical structure.

In [ ]:
spec = [
    ['Country'],
    ['Country', 'State'], 
    ['Country', 'Purpose'], 
    ['Country', 'State', 'Region'], 
    ['Country', 'State', 'Purpose'], 
    ['Country', 'State', 'Region', 'Purpose']
]

Using the `aggregate` function from `HierarchicalForecast` we can get the full set of time series.

In [ ]:
from hierarchicalforecast.utils import aggregate

In [ ]:
Y_df_cs, S_df_cs, tags_cs = aggregate(Y_df, spec)

In [ ]:
Y_df_cs

,unique_id,ds,y
0,Australia,1998-01-01,23182.197269
1,Australia,1998-04-01,20323.380067
2,Australia,1998-07-01,19826.640511
3,Australia,1998-10-01,20830.129891
4,Australia,1999-01-01,22087.353380
...,...,...,...
33995,Australia/Western Australia/Experience Perth/V...,2016-10-01,439.699451
33996,Australia/Western Australia/Experience Perth/V...,2017-01-01,356.867038
33997,Australia/Western Australia/Experience Perth/V...,2017-04-01,302.296119
33998,Australia/Western Australia/Experience Perth/V...,2017-07-01,373.442070


In [ ]:
S_df_cs.iloc[:5, :5]

,unique_id,Australia/ACT/Canberra/Business,Australia/ACT/Canberra/Holiday,Australia/ACT/Canberra/Other,Australia/ACT/Canberra/Visiting
0,Australia,1.0,1.0,1.0,1.0
1,Australia/ACT,1.0,1.0,1.0,1.0
2,Australia/New South Wales,0.0,0.0,0.0,0.0
3,Australia/Northern Territory,0.0,0.0,0.0,0.0
4,Australia/Queensland,0.0,0.0,0.0,0.0


### 2b. Split Train/Test sets

We use the final two years (8 quarters) as test set. Consequently, our forecast horizon=8.

In [ ]:
horizon = 8

In [ ]:
Y_test_df_cs = Y_df_cs.groupby("unique_id", as_index=False).tail(horizon)
Y_train_df_cs = Y_df_cs.drop(Y_test_df_cs.index)

### 2c. Computing base forecasts

The following cell computes the **base forecasts** for each time series in `Y_df` using the `ETS` model. Observe that `Y_hat_df` contains the forecasts but they are not coherent.

In [ ]:
from statsforecast.models import AutoETS
from statsforecast.core import StatsForecast

In [ ]:
fcst = StatsForecast(models=[AutoETS(season_length=4, model='ZZA')], 
                     freq='QS', n_jobs=-1)
Y_hat_df_cs = fcst.forecast(df=Y_train_df_cs, h=horizon, fitted=True)
Y_fitted_df_cs = fcst.forecast_fitted_values()

### 2d. Reconcile forecasts

The following cell makes the previous forecasts coherent using the `HierarchicalReconciliation` class. Since the hierarchy structure is not strict, we can't use methods such as `TopDown` or `MiddleOut`. In this example we use `BottomUp` and `MinTrace`.

In [ ]:
from hierarchicalforecast.methods import BottomUp, MinTrace
from hierarchicalforecast.core import HierarchicalReconciliation

In [ ]:
reconcilers = [
    BottomUp(),
    MinTrace(method='mint_shrink'),
    MinTrace(method='ols')
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df_cs = hrec.reconcile(Y_hat_df=Y_hat_df_cs, Y_df=Y_fitted_df_cs, S=S_df_cs, tags=tags_cs)

The dataframe `Y_rec_df` contains the reconciled forecasts.

In [ ]:
Y_rec_df_cs.head()

,unique_id,ds,AutoETS,AutoETS/BottomUp,AutoETS/MinTrace_method-mint_shrink,AutoETS/MinTrace_method-ols
0,Australia,2016-01-01,25990.068004,24381.911737,25428.089783,25894.399067
1,Australia,2016-04-01,24458.490282,22903.895964,23914.271400,24357.301898
2,Australia,2016-07-01,23974.055984,22412.265739,23428.462394,23865.910647
3,Australia,2016-10-01,24563.454495,23127.349578,24089.845955,24470.782393
4,Australia,2017-01-01,25990.068004,24518.118006,25545.358678,25901.362283


## 3. Temporal reconciliation

### 3a. Aggregating the dataset according to temporal hierarchy

In [ ]:
from hierarchicalforecast.utils import aggregate_temporal

We first define the temporal aggregation spec. You can use string aliases of timestamp attributes to compute temporal aggregations. For Pandas, see an overview of allowable attributes [here](https://pandas.pydata.org/docs/reference/api/pandas.Timestamp.html). For Polars

In [ ]:
spec_temporal = [
    ["year"],
    ["year", "ds"],
]   

We next compute the temporal aggregated train- and test sets. Note that we have different aggregation matrices S for the train- and test set.

In [ ]:
Y_df_te_train, S_df_te_train, tags_te_train = aggregate_temporal(df=Y_train_df_cs, freq="QS", spec=spec_temporal)
Y_df_te_test, S_df_te_test, tags_te_test = aggregate_temporal(df=Y_test_df_cs, freq="QS", spec=spec_temporal)


If you don't have a test set available, as is usually the case when you're making forecasts, it is necessary to create a future dataframe that holds the correct bottom-level unique_ids and timestamps so that they can be temporally aggregated. This can be done as follows.

In [ ]:
from hierarchicalforecast.utils import make_future_dataframe

In [ ]:
Y_test_df_cs_new = make_future_dataframe(Y_df_te_train, freq="QS", h=horizon)

`Y_test_df_cs_new` can be then used in `aggregate_temporal` to construct the temporally aggregated structures:

In [ ]:
Y_df_te_test_new, S_df_te_test_new, tags_te_test_new = aggregate_temporal(df=Y_test_df_cs_new, freq="QS", spec=spec_temporal)


And we can verify that we have the same temporally aggregated test set, except that `Y_df_te_test_new` doesn't contain the ground truth values `y`.

In [ ]:
Y_df_te_test

,temporal_id,unique_id,y,ds
0,2016,Australia,101484.586551,2016-01-01
1,2016,Australia/ACT,2457.401367,2016-01-01
2,2016,Australia/ACT/Business,754.139245,2016-01-01
3,2016,Australia/ACT/Canberra,2457.401367,2016-01-01
4,2016,Australia/ACT/Canberra/Business,754.139245,2016-01-01
...,...,...,...,...
4245,2017/2017-10-01,Australia/Western Australia/Experience Perth/O...,87.494916,2017-10-01
4246,2017/2017-10-01,Australia/Western Australia/Experience Perth/V...,455.316702,2017-10-01
4247,2017/2017-10-01,Australia/Western Australia/Holiday,1026.285985,2017-10-01
4248,2017/2017-10-01,Australia/Western Australia/Other,161.087339,2017-10-01


In [ ]:
Y_df_te_test_new

,temporal_id,unique_id,ds
0,2016,Australia,2016-01-01
1,2016,Australia/ACT,2016-01-01
2,2016,Australia/ACT/Business,2016-01-01
3,2016,Australia/ACT/Canberra,2016-01-01
4,2016,Australia/ACT/Canberra/Business,2016-01-01
...,...,...,...
4245,2017/2017-10-01,Australia/Western Australia/Experience Perth/O...,2017-10-01
4246,2017/2017-10-01,Australia/Western Australia/Experience Perth/V...,2017-10-01
4247,2017/2017-10-01,Australia/Western Australia/Holiday,2017-10-01
4248,2017/2017-10-01,Australia/Western Australia/Other,2017-10-01


### 3b. Computing base forecasts

The following cell computes the **base forecasts** for each temporal aggregation in `Y_dfs_te` using the `AutoETS` model. Observe that `Y_hat_df_te` contains the forecasts but they are not coherent.

In [ ]:
Y_hat_dfs_te = []
# We will train a model for each temporal level
for level, temporal_ids_train in tags_te_train.items():
    # Filter the data for the level
    Y_level_train = Y_df_te_train.query("temporal_id in @temporal_ids_train")
    temporal_ids_test = tags_te_test[level]
    Y_level_test = Y_df_te_test.query("temporal_id in @temporal_ids_test")
    # For each temporal level we have a different frequency and forecast horizon
    freq_level = pd.infer_freq(Y_level_train["ds"].unique())
    horizon_level = Y_level_test["ds"].nunique()
    # Train a model and create forecasts
    fcst = StatsForecast(models=[AutoETS(model='ZZZ')], freq=freq_level, n_jobs=-1)
    Y_hat_df_te_level = fcst.forecast(df=Y_level_train[["ds", "unique_id", "y"]], h=horizon_level)
    # Add the test set to the forecast
    Y_hat_df_te_level = Y_hat_df_te_level.merge(Y_level_test, on=["ds", "unique_id"], how="left")
    Y_hat_df_te_level = Y_hat_df_te_level[["unique_id", "temporal_id", "ds", "y", "AutoETS"]]
    Y_hat_dfs_te.append(Y_hat_df_te_level)

Y_hat_df_te = pd.concat(Y_hat_dfs_te, ignore_index=True)


### 3c. Reconcile forecasts

The following cell makes the previous forecasts coherent using the `HierarchicalReconciliation` class. In this example we use `BottomUp` and `MinTrace`. 

Note that temporal reconcilation currently isn't supported for insample reconciliation methods, such as `MinTrace(method='mint_shrink')`.

In [ ]:
reconcilers = [
    BottomUp(),
    MinTrace(method='ols')
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df_te = hrec.reconcile(Y_hat_df=Y_hat_df_te, S=S_df_te_test, tags=tags_te_test, temporal=True)

## 4. Evaluation 

The `HierarchicalForecast` package includes the `HierarchicalEvaluation` class to evaluate the different hierarchies and also is capable of compute scaled metrics compared to a benchmark model.

In [ ]:
from hierarchicalforecast.evaluation import evaluate
from utilsforecast.losses import rmse, mase
from functools import partial

### 4a. Cross-sectional evaluation

We first evaluate the cross-sectional forecasts _across all temporal aggregations_.

In [ ]:
eval_tags = {}
eval_tags['Total'] = tags_cs['Country']
eval_tags['Purpose'] = tags_cs['Country/Purpose']
eval_tags['State'] = tags_cs['Country/State']
eval_tags['Regions'] = tags_cs['Country/State/Region']
eval_tags['Bottom'] = tags_cs['Country/State/Region/Purpose']

evaluation = evaluate(df = Y_rec_df_te.drop(columns='temporal_id'),
                      tags = eval_tags,
                      metrics = [rmse])

evaluation.columns = ['level', 'metric', 'Base', 'BottomUp', 'MinTrace(ols)']
numeric_cols = evaluation.select_dtypes(include="number").columns
evaluation[numeric_cols] = evaluation[numeric_cols].map('{:.2f}'.format).astype(np.float64)

In [ ]:
print(evaluation)

     level metric     Base  BottomUp  MinTrace(ols)
0    Total   rmse  4089.09   4482.04        4083.23
1  Purpose   rmse  1167.85   1284.27        1114.90
2    State   rmse   631.81    540.32         614.16
3  Regions   rmse   100.14    105.58          96.51
4   Bottom   rmse    32.39     33.34          31.62
5  Overall   rmse    79.58     81.68          77.37


### 4b. Temporal evaluation

We then evaluate the temporally aggregated forecasts _across all cross-sections_.

In [ ]:
eval_tags = {}
eval_tags['Year'] = tags_te_test['year']
eval_tags['Quarter'] = tags_te_test['year/ds']

evaluation = evaluate(df = Y_rec_df_te.drop(columns='unique_id'),
                      tags = eval_tags,
                      id_col = 'temporal_id',
                      metrics = [rmse])

In [ ]:
print(evaluation)

     level metric     AutoETS  AutoETS/BottomUp  AutoETS/MinTrace_method-ols
0     Year   rmse  480.851970        581.181898                   491.843641
1  Quarter   rmse  168.016503        168.016503                   152.299021
2  Overall   rmse  230.583597        250.649582                   220.207945


### 4c. Cross-temporal evaluation

Finally, we evaluate cross-temporally.


In [ ]:
import itertools

In [ ]:
tags_ct = {}
for key_cs, value_cs in tags_cs.items():
    for key_te, value_te in tags_te_test.items():
        key_ct = key_cs + "//" + key_te
        value_ct = list("//".join(s) for s in itertools.product(value_cs, value_te))
        tags_ct[key_ct] = value_ct

In [ ]:
Y_rec_df_te["cross_temporal_id"] = Y_rec_df_te["unique_id"] + "//" + Y_rec_df_te["temporal_id"]

In [ ]:
eval_tags = {}
eval_tags['TotalByYear'] = tags_ct['Country//year']
eval_tags['TotalByQuarter'] = tags_ct['Country//year/ds']
eval_tags['RegionsByYear'] = tags_ct['Country/State/Region//year']
eval_tags['RegionsByQuarter'] = tags_ct['Country/State/Region//year/ds']
eval_tags['BottomByYear'] = tags_ct['Country/State/Region/Purpose//year']
eval_tags['BottomByQuarter'] = tags_ct['Country/State/Region/Purpose//year/ds']

evaluation = evaluate(df = Y_rec_df_te.drop(columns=['unique_id', 'temporal_id']),
                      tags = eval_tags,
                      id_col = 'cross_temporal_id',
                      metrics = [rmse])

In [ ]:
print(evaluation)

              level metric      AutoETS  AutoETS/BottomUp  \
0       TotalByYear   rmse  7148.989568       8243.064021   
1    TotalByQuarter   rmse  2060.766005       2060.766005   
2     RegionsByYear   rmse   151.959299        175.694257   
3  RegionsByQuarter   rmse    57.073162         57.073162   
4      BottomByYear   rmse    46.979437         50.784616   
5   BottomByQuarter   rmse    19.416658         19.416658   
6           Overall   rmse    43.141001         45.269455   

   AutoETS/MinTrace_method-ols  
0                  7367.804458  
1                  1876.541785  
2                   153.940701  
3                    53.986267  
4                    46.898020  
5                    18.743656  
6                    42.012900  


We find that the best method is the cross-temporally reconciled method `AutoETS/MinTrace_method-ols`, which achieves overall lowest RMSE.

### Comparison fable

Observe that we can recover the results reported by the [Forecasting: Principles and Practice](https://otexts.com/fpp3/tourism.html). The original results were calculated using the R package [fable](https://github.com/tidyverts/fable).

![Fable's reconciliation results](./imgs/AustralianDomesticTourism-results-fable.png)

### References
- [Hyndman, R.J., & Athanasopoulos, G. (2021). "Forecasting: principles and practice, 3rd edition: 
Chapter 11: Forecasting hierarchical and grouped series.". OTexts: Melbourne, Australia. OTexts.com/fpp3 
Accessed on July 2022.](https://otexts.com/fpp3/hierarchical.html)
- [Rob Hyndman, Alan Lee, Earo Wang, Shanika Wickramasuriya, and Maintainer Earo Wang (2021). "hts: Hierarchical and Grouped Time Series". URL https://CRAN.R-project.org/package=hts. R package version 0.3.1.](https://cran.r-project.org/web/packages/hts/index.html)
- [Mitchell O’Hara-Wild, Rob Hyndman, Earo Wang, Gabriel Caceres, Tim-Gunnar Hensel, and Timothy Hyndman (2021). "fable: Forecasting Models for Tidy Time Series". URL https://CRAN.R-project.org/package=fable. R package version 6.0.2.](https://CRAN.R-project.org/package=fable)